---

<center> <h1> Transformation of Olympic Athletes Dataset </h1> </center>

---

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Basic Libraries
import pandas as pd

**This part of the notebook will focus on:**
1. Bringing in the world GDP dataset.
2. Bringing in the world population dataset.
3. Merging all 3 datasets (including original cleaned athlete dataset).

In [3]:
#Import cleaned athlete data csv file into notebook
athleteDatas = pd.read_csv('Cleaned Data/athleteDatas.csv')
athleteDatas.head()

,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,Season,City,Sport,Event,Medal,Team
0,1,A Dijiang,M,24.0,180.0,80.0,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,China
1,2,A Lamusi,M,23.0,170.0,60.0,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,China
2,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,Netherlands
3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",No Medal,Netherlands
4,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,No Medal,Netherlands


**First we bring in the world GDP data.**

In [4]:
#Importing GDP data process
#Data will be used for analyzing total medals per country later on

# Glance at the data.
gdp = pd.read_csv('World Data/world_gdp.csv', skiprows = 3)

# Remove unnecessary columns
gdp.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace = True)

# The columns are the years for which the GDP has been recorded. 
# This needs to brought into a single column for efficient merging.
gdp = pd.melt(gdp, id_vars = ['Country Name', 'Country Code'], var_name = 'Year', value_name = 'GDP')

# convert the year column to numeric
gdp['Year'] = pd.to_numeric(gdp['Year'])

gdp.head()

,Country Name,Country Code,Year,GDP
0,Afghanistan,AFG,1960,5.377778e+08
1,Albania,ALB,1960,NaN
2,Algeria,DZA,1960,2.723649e+09
3,American Samoa,ASM,1960,NaN
4,Andorra,AND,1960,NaN


In [5]:
# Merge to get country code
olympics_merge_ccode = athleteDatas.merge(gdp[['Country Name', 'Country Code']].drop_duplicates(),
                                            left_on = 'Team',
                                            right_on = 'Country Name',
                                            how = 'left')

olympics_merge_ccode.drop('Country Name', axis = 1, inplace = True)

# Merge to get gdp too
olympics_merge_gdp = olympics_merge_ccode.merge(gdp,
                                                left_on = ['Country Code', 'Year'],
                                                right_on = ['Country Code', 'Year'],
                                                how = 'left')

olympics_merge_gdp.drop('Country Name', axis = 1, inplace = True)
olympics_merge_gdp.head(3)

,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,Season,City,Sport,Event,Medal,Team,Country Code,GDP
0,1,A Dijiang,M,24.0,180.0,80.0,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,China,CHN,4.269160e+11
1,2,A Lamusi,M,23.0,170.0,60.0,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,China,CHN,8.560550e+12
2,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,Netherlands,NLD,2.585680e+11


***

**Now we can bring in the world population data.**

In [6]:
# Read in the population data
pop = pd.read_csv('World Data/world_pop.csv')

pop.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace = True)

pop = pd.melt(pop, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population')

# Change the Year to integer type
pop['Year'] = pd.to_numeric(pop['Year'])

pop.head(3)

,Country,Country Code,Year,Population
0,Aruba,ABW,1960,54211.0
1,Afghanistan,AFG,1960,8996351.0
2,Angola,AGO,1960,5643182.0


In [7]:
#Merging our main data set with the population data
athleteComplete = olympics_merge_gdp.merge(pop,
                                            left_on = ['Country Code', 'Year'],
                                            right_on= ['Country Code', 'Year'],
                                            how = 'left')

athleteComplete.drop('Country', axis = 1, inplace = True)

athleteComplete.head(3)

,ID,Name,Sex,Age,Height,Weight,NOC,Games,Year,Season,City,Sport,Event,Medal,Team,Country Code,GDP,Population
0,1,A Dijiang,M,24.0,180.0,80.0,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,China,CHN,4.269160e+11,1.164970e+09
1,2,A Lamusi,M,23.0,170.0,60.0,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,China,CHN,8.560550e+12,1.350695e+09
2,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,Netherlands,NLD,2.585680e+11,1.476009e+07


**Population and GDP are only for 1961 onwards while Olympics data is from 1896. 
<br> Therefore, let's consider only data from 1961 onwards.**

---

<center> <h2> Done with transforming dataset by merging with GDP & Population </h2> </center>

---

In [8]:
#Export cleaned dataset for data persistence
athleteComplete.to_csv('Cleaned Data/athleteComplete.csv', index=False)

---

## Reference materials & Sources: 

1. https://chrisalbon.com/python/data_wrangling/
2. Data for Athletes for most appearance (Top 50) from: https://en.wikipedia.org/wiki/List_of_athletes_with_the_most_appearances_at_Olympic_Games
3. https://www.theanalysisfactor.com/regression-modelshow-do-you-know-you-need-a-polynomial/


---